In [ ]:
# import socket
# import json
# import pandas as pd
# import time

In [ ]:
# def send_data_over_socket(file_path, host='localhost', port=9999, chunk_size=2):
#     s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
#     s.bind((host, port))
#     s.listen(1)
#     print(f"Listening for connections on {host}:{port}")
    
#     conn, addr = s.accept()
#     print(f"Connection from {addr}")
    
#     last_sent_index = 0
#     try:
#         with open(file_path, 'r') as file:
#             # skip the lines that were already sent
#             for _ in range(last_sent_index):
#                 next(file)
            
#             records = []
#             for line in file:
#                 records.append(json.loads(line))
#                 if len(records) == chunk_size:
#                     chunk = pd.Dataframe(records)
#                     print(chunk)
#                     for record in chunk.to_dict(orient='records'):
#                         serialize_data = json.dumps(record).encode('utf-8')
#                         conn.send(serialize_data + b'\n')
#                         time.sleep(5)
#                         last_sent_index += 1
                    
#                     records = []
#     except (BrokenPipeError, ConnectionResetError):
#         print('Client disconnected.')
#     finally:
#         conn.close()
#         print('Connection closed')

In [ ]:
# send_data_over_socket('datasets/yelp_academic_dataset_review.json')

In [ ]:
# import openai
# from config.config import config

# comment = 'Your food is ... i do not know. maybe not good and not bad at the same'

# openai.api_key = config['openai']['api_key']
# completion = openai.ChatCompletion.create(
#     model='gpt-3.5-turbo-1106',
#     messages = [
#         {
#             "role": "system",
#             "content": f"""
#                 You're a machine learning model with a task of classifying comments into POSITIVE, NEGATIVE, NEUTRAL.
#                 You are to respond with one word from the option specified above, do not add anything else.
#                 Here is the comment: {comment}
#             """
#         }
#     ]
# )
# completion.choices[0].message['content']

In [ ]:
import openai
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark_conn = SparkSession.builder.appName("SocketStreamConsumer").getOrCreate()

def start_streaming(spark):
    try:
        stream_df = (spark.readStream.format("socket")
                .option("host", "localhost")
                .option("port", 9999)
                .load()
                )

        schema = StructType([
            StructField("review_id", StringType()),
            StructField("user_id", StringType()),
            StructField("business_id", StringType()),
            StructField("stars", FloatType()),
            StructField("date", StringType()),
            StructField("text", StringType())
        ])

        stream_df = stream_df.select(from_json(col('value'), schema).alias("data")).select(("data.*"))

        query = stream_df.writeStream.outputMode("append").format("console").start()
        query.awaitTermination()
        # sentiment_analysis_udf = udf(sentiment_analysis, StringType())

        # stream_df = stream_df.withColumn('feedback',
        #                                     when(col('text').isNotNull(), sentiment_analysis_udf(col('text')))
        #                                     .otherwise(None)
        #                                     )
    except Exception as e:
        print(e)
        
start_streaming(spark_conn)